In [10]:
# Import libraries
import tensorflow.compat.v1 as tf
import numpy as np

In [2]:
# Generate data points

class_A = np.random.rand(1000,18)

Class_B = np.random.rand(1000,18)

In [4]:
# Combine the classes to generate dataset
dataset = np.vstack([class_A, Class_B])

In [7]:
# set the labels
labels = np.vstack([np.ones((len(class_A),1)),np.zeros((len(Class_B),1))])

In [6]:
# Check dataset shape
dataset.shape

(2000, 18)

In [8]:
#Check label shape
labels.shape

(2000, 1)

In [15]:
# Define placeholder for the support set and query set
tf.compat.v1.disable_eager_execution()
x_i = tf.placeholder(tf.float32, [None, 9])
x_j = tf.placeholder(tf.float32, [None, 9])

In [13]:
# Define placeholder for label
y = tf.placeholder(tf.float32, [None, 1])

In [14]:
# Define embedding function which with learn embeddings from query set and support set.
def embedding_function(x):
    weight = tf.Variable(tf.truncated_normal([9,1]))
    bias = tf.Variable(tf.truncated_normal([1]))    
    a = (tf.nn.xw_plus_b(x,weight,bias))
    return tf.nn.relu(a)

In [18]:
#Compute embedding for support set
f_Xi = embedding_function(x_i)

In [19]:
# Compute embedding for Query set
f_Xj = embedding_function(x_j)

In [20]:
# Combine support set and query set features
Z = tf.concat([f_Xi,f_Xj],axis=1)

In [22]:
# Define Relational function
def relational_function(x):
    w_1 = tf.Variable(tf.truncated_normal([2,3]))
    b_1 = tf.Variable(tf.truncated_normal([3]))
    
    w_2 = tf.Variable(tf.truncated_normal([3,5]))
    b_2 = tf.Variable(tf.truncated_normal([5]))
    
    w_3 = tf.Variable(tf.truncated_normal([5,1]))
    b_3 = tf.Variable(tf.truncated_normal([1]))
    
    #layer1
    z_1 = (tf.nn.xw_plus_b(x,w_1,b_1))
    a_1 = tf.nn.relu(z_1)
    
    #layer2
    z_2 = tf.nn.xw_plus_b(a_1,w_2,b_2)
    a_2 = tf.nn.relu(z_2)
    
    #layer3
    z_3 = tf.nn.xw_plus_b(z_2,w_3,b_3)

    #output
    y = tf.nn.sigmoid(z_3)
    
    return y

In [24]:
# pass feature vector
relational_scores = relational_function(Z)

In [25]:
# Define loss function as a mean sequered error between label and relational score
loss_function = tf.reduce_mean(tf.squared_difference(relational_scores,y))

In [26]:
#Loss minimization
optimizer = tf.train.AdamOptimizer(0.1)
train = optimizer.minimize(loss_function)

In [27]:
# Start tensorflow session
session = tf.InteractiveSession()
session.run(tf.global_variables_initializer())

In [34]:
# Train the relational network
for episode in range(1000):
    _, loss_value = session.run([train, loss_function], 
                             feed_dict={x_i:dataset[:,0:9]+np.random.randn(*np.shape(dataset[:,0:9]))*0.05,
                                        x_j:dataset[:,9:]+np.random.randn(*np.shape(dataset[:,9:]))*0.05,
                                        y:label})
    if episode % 100 == 0:
        print("Episode {}: loss {:.3f} ".format(episode, loss_value))

Episode 0: loss 0.330 
Episode 100: loss 0.250 
Episode 200: loss 0.250 
Episode 300: loss 0.250 
Episode 400: loss 0.250 
Episode 500: loss 0.250 
Episode 600: loss 0.250 
Episode 700: loss 0.250 
Episode 800: loss 0.250 
Episode 900: loss 0.250 
